In [1]:
import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt
# import scipy.stats as stats

# Set random seed for reproducibility
np.random.seed(42)

# Generate data for 10 individuals
n = 1000

# Simulate market income (y)
income = np.random.lognormal(mean=10, sigma=0.8, size=n)

# Simulate household size 
household_size = np.random.randint(1, 6, size=n)

# Simulate number of dependents (a)
dependents = np.random.binomial(household_size - 1, 0.5, size=n)
dependents = np.clip(dependents, 0, household_size - 1)

# Households with size 1 have 0 dependents
dependents[household_size == 1] = 0

# Calculate working adults (A)
working = household_size - dependents

# Simulate weights
weights = np.random.uniform(0.5, 1.5, size=n)

# Simulate taxes (T)
tau = 0.6
theta = 0.8
ell = 100
sigma = 0.5
net_income = (
    pow(household_size, theta) / working * (
        ell * pow(income, 1 - tau) * (
            np.exp(np.random.normal(scale = sigma, size = n))
            )
        )
)

# Create DataFrame
df = pd.DataFrame({
    'income': income,  # Market income
    'net_income': net_income,  # Net income after taxes
    'working': working,  # Household size
    'household_size': household_size,  # Non-dependents in household
    'weights': weights,  # Survey weights
})


# Filter the data
df = df[(df['income'] > 0) & (df['net_income'] > 0)] 


# Display the data
df

,income,net_income,working,household_size,weights
0,32773.361742,5828.091824,4,4,1.041990
1,19719.998637,2817.568018,1,1,1.439652
2,36980.677685,3270.696039,3,3,0.720484
3,74490.276951,7152.389958,4,5,1.225726
4,18263.819407,4230.106960,3,3,1.117249
...,...,...,...,...,...
995,17590.596803,5334.819696,2,3,0.751080
996,92795.109820,3542.866326,4,4,1.150425
997,36778.705030,4846.601972,1,1,0.712334
998,13947.517413,9672.465028,2,3,0.622007


In [2]:
import monte_carlo as m

df = m.simulate_hsv_sample(1000, 0.6, 0.8, 100, 0.5)
df

,income,net_income,working,household_size
0,35288.032336,5577.943616,1,1
1,25206.234433,12473.691686,2,4
2,38870.955568,13420.824391,2,4
3,94316.252170,6003.809404,2,2
4,35306.599511,5170.145086,3,4
...,...,...,...,...
995,16684.612931,6514.159602,4,4
996,26736.837821,4993.522568,3,5
997,4736.810972,4126.273513,3,4
998,56697.617560,8187.049742,1,1


In [ ]:
help(np.exp)

## OLS

In [4]:
import statsmodels.api as sm

X = np.log(df[['income', 'household_size']])
X = sm.add_constant(X)
# y = np.log(df['net_income'] / df['working'])
y = np.log(df['net_income']) + np.log(df['working'])

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.548
Method:                 Least Squares   F-statistic:                     606.8
Date:                Fri, 21 Mar 2025   Prob (F-statistic):          4.13e-173
Time:                        11:51:56   Log-Likelihood:                -746.05
No. Observations:                1000   AIC:                             1498.
Df Residuals:                     997   BIC:                             1513.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.6247      0.209     22.153      0.000       4.215       5.034
income             0.3967      0.021     19.213      0.000       0.356       0.437
household_size     0.8152      0.028     29.375      0.000       0.761       0.870
==============================================================================
Omnibus:                        0.449   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.799   Jarque-Bera (JB):                0.535
Skew:                           0.033   Prob(JB):                        0.765
Kurtosis:                       2.907   Cond. No.                         131.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# Residual estimate
pow(np.var(model.resid), 1/2)

0.5102344512720337

## Maximum Likelihood

In [6]:
# Split dataframe

# // is the floor division operator
half_index = df.shape[0] // 2

df_2 = pd.concat([
    # .iloc[:x] selects all rows from the beginning up to, but not including, the row at position x
        df.iloc[:half_index], 
        df.iloc[half_index:].reset_index(drop = True)
    ], axis=1)

df_2

,income,net_income,working,household_size,income,net_income,working,household_size
0,35288.032336,5577.943616,1,1,16535.054397,3326.245037,1,1
1,25206.234433,12473.691686,2,4,31561.288407,10080.146278,3,5
2,38870.955568,13420.824391,2,4,12066.351577,4933.534909,2,4
3,94316.252170,6003.809404,2,2,25741.092711,10449.057767,2,3
4,35306.599511,5170.145086,3,4,60323.540875,8934.756047,2,3
...,...,...,...,...,...,...,...,...
495,15038.478178,7914.068924,1,1,16684.612931,6514.159602,4,4
496,13118.358813,10576.834327,1,2,26736.837821,4993.522568,3,5
497,9408.351184,7163.263662,1,2,4736.810972,4126.273513,3,4
498,24858.218831,9297.251180,2,3,56697.617560,8187.049742,1,1


In [7]:
column_names = df.columns

new_column_names = []
for k in column_names:
    new_name = str(k) + "_i"
    new_column_names.append(new_name)
for k in column_names:
    new_name = str(k) + "_j"
    new_column_names.append(new_name)

df_2.columns = new_column_names

df_2

,income_i,net_income_i,working_i,household_size_i,income_j,net_income_j,working_j,household_size_j
0,35288.032336,5577.943616,1,1,16535.054397,3326.245037,1,1
1,25206.234433,12473.691686,2,4,31561.288407,10080.146278,3,5
2,38870.955568,13420.824391,2,4,12066.351577,4933.534909,2,4
3,94316.252170,6003.809404,2,2,25741.092711,10449.057767,2,3
4,35306.599511,5170.145086,3,4,60323.540875,8934.756047,2,3
...,...,...,...,...,...,...,...,...
495,15038.478178,7914.068924,1,1,16684.612931,6514.159602,4,4
496,13118.358813,10576.834327,1,2,26736.837821,4993.522568,3,5
497,9408.351184,7163.263662,1,2,4736.810972,4126.273513,3,4
498,24858.218831,9297.251180,2,3,56697.617560,8187.049742,1,1


In [8]:
# Add an indicator variable

df_2 = df_2.assign(
    atr_i = df_2['net_income_i'] / df_2['income_i'],
    atr_j = df_2['net_income_j'] / df_2['income_j'],
)

df_2 = df_2.assign(
    rank_binary = np.where(df_2['atr_i'] < df_2['atr_j'], 1, 0)
)

# df_2["rank_binary"] = np.where(
#     (df_2['net_income_i'] / df_2['income_i'] 
#      > df_2['net_income_j'] / df_2['income_j']), 
#      1, 0
# )

df_2

,income_i,net_income_i,working_i,household_size_i,income_j,net_income_j,working_j,household_size_j,atr_i,atr_j,rank_binary
0,35288.032336,5577.943616,1,1,16535.054397,3326.245037,1,1,0.158069,0.201163,1
1,25206.234433,12473.691686,2,4,31561.288407,10080.146278,3,5,0.494865,0.319383,0
2,38870.955568,13420.824391,2,4,12066.351577,4933.534909,2,4,0.345266,0.408867,1
3,94316.252170,6003.809404,2,2,25741.092711,10449.057767,2,3,0.063656,0.405929,1
4,35306.599511,5170.145086,3,4,60323.540875,8934.756047,2,3,0.146436,0.148114,1
...,...,...,...,...,...,...,...,...,...,...,...
495,15038.478178,7914.068924,1,1,16684.612931,6514.159602,4,4,0.526255,0.390429,0
496,13118.358813,10576.834327,1,2,26736.837821,4993.522568,3,5,0.806262,0.186766,0
497,9408.351184,7163.263662,1,2,4736.810972,4126.273513,3,4,0.761373,0.871108,1
498,24858.218831,9297.251180,2,3,56697.617560,8187.049742,1,1,0.374011,0.144398,0


In [9]:
from scipy.stats import norm
import scipy.optimize as optim



# Specify a likelihood function
def neg_log_likelihood(params, data):
      tau, theta, sigma = params
      
      # Specify random variable
      randvar = (
            tau * np.log(data['net_income_i'] / data['net_income_j'])
            + np.log(data['working_i'] / data['working_j'])
            - theta * (
                  np.log(data['household_size_i'] / data['household_size_j'])
            )
      )
      # Specify CDF
      cdf = norm.cdf(randvar, scale = np.pow(sigma, 1/2))
      # Specify log-likelihood function
      log_likelihood = np.sum(
        data['rank_binary'] * np.log(cdf) 
        + (1 - data['rank_binary']) * np.log(1 - cdf)
      )
      return -log_likelihood

initial_params = [0.6, 0.8, 0.5]

model = optim.minimize(
      neg_log_likelihood, initial_params, args = (df_2,),
      method='BFGS'
      )

model.x

array([-0.52669941,  0.98304851,  2.23777182])

In [ ]:
# help(norm.cdf)
help(optim.minimize)